In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import yaml
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import random

from src.data.prepare_data import prepare_data
from src.models.utils import train_splits, imbalanced_sampling
from src.models.tuner import HyperParamSearch, LabelWeightSearch
from src.models.model import Classifier
from src.models.evaluation import Evaluation

# ensure reproducibility
seed=42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
df = pd.read_csv('../' + config['data_loader']['path'])
df = prepare_data(df=df)
display(df.head())

# check class distributions
df['label'].value_counts(normalize=True)

,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v21,v22,v23,v24,v25,v26,v27,v28,amount,label
0,-1.996580,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,-1.996580,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342474,0
2,-1.996558,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160684,0
3,-1.996558,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,-1.996537,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


label
0    0.998273
1    0.001727
Name: proportion, dtype: float64

**Train and Test Set**
- Split whole set into train, validation and test sets using strat sampling
- Apply oversampling as the number of positive instances is small

In [4]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df['label']
    , test_size=config['train_test_split']['test_size']
    , random_state=123
    , shuffle=True
    , stratify=df['label']
    )

# split dataset into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train
    , test_size=config['train_test_split']['test_size']
    , random_state=123
    , shuffle=True
    , stratify=y_train
    )

In [5]:
# apply oversampling to the train set
X_train_rs, y_train_rs = imbalanced_sampling(
    method='over'
    , X_train=X_train
    , y_train=y_train
)

# check class distributions
print(
    y_train.value_counts(normalize=True)
    , y_train_rs.value_counts(normalize=True)
    )

label
0    0.998273
1    0.001727
Name: proportion, dtype: float64 label
0    0.5
1    0.5
Name: proportion, dtype: float64


**Optimization & Evaluation**
- Hyper-parameters - search which hyper-parameters optimize scoring metric for the given algorithm in the validation set
- Evaluation - retrieve best hyper-parameters and recover full training set, applying over sampling, to evaluate results on test set

In [6]:
ALGORITHM = "NeuralNetworkClassifier"

In [7]:
# utils function in case ALGORITHM is NeuralNetworkClassifier
def model_builder(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(30,)))

    n_layers = hp.Int("n_layers", min_value=4, max_value=8, step=1)
    for i in range(n_layers):
        hp_units = hp.Int(f"units_{i}", min_value=32, max_value=128, step=4)
        hp_activation = hp.Choice(f"activation_{i}", values=["sigmoid", "relu"])
        model.add(keras.layers.Dense(units=hp_units, activation=hp_activation))
    # output layer
    model.add(keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
        )
    
    return model

In [8]:
if ALGORITHM != 'NeuralNetworkClassifier':

    # set tuner for hyperparam optimization
    tuner = HyperParamSearch(config=config['optimization'], algorithm=ALGORITHM)

    def objective(trial) -> float:
        return tuner.fit(
            X=X_train_rs, y=y_train_rs, trial=trial,
            X_val=X_val, y_val=y_val
            )

    # set study
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=10)

    print(
        f"Best trial:\n",
        f"- params: {study.best_trial.params}\n",
        f"- score: {study.best_trial.value}", "\n"*2
    )

else:
    bayesian_search_tuner = kt.BayesianOptimization(
        hypermodel=model_builder,
        objective='val_accuracy',
        max_trials=2,
        seed=42,
        overwrite=False
        #, directory=
        , project_name="keras_tuner"
        )

    bayesian_search_tuner.search(
        X_train_rs.to_numpy(), y_train_rs.to_numpy(),
        epochs=5, batch_size=32, validation_data=(X_val, y_val)
    )

    print(bayesian_search_tuner.results_summary())

Trial 2 Complete [00h 03m 28s]
val_accuracy: 0.9991869330406189

Best val_accuracy So Far: 0.9991869330406189
Total elapsed time: 00h 06m 31s
Results summary
Results in .\keras_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
n_layers: 5
units_0: 128
activation_0: relu
units_1: 120
activation_1: sigmoid
units_2: 128
activation_2: sigmoid
units_3: 88
activation_3: sigmoid
units_4: 60
activation_4: sigmoid
units_5: 124
activation_5: sigmoid
units_6: 76
activation_6: sigmoid
Score: 0.9991869330406189

Trial 0 summary
Hyperparameters:
n_layers: 7
units_0: 32
activation_0: sigmoid
units_1: 56
activation_1: relu
units_2: 64
activation_2: relu
units_3: 36
activation_3: sigmoid
units_4: 32
activation_4: sigmoid
units_5: 32
activation_5: sigmoid
units_6: 32
activation_6: sigmoid
Score: 0.9986695051193237
None


In [9]:
# fit model on whole training set
#X_train, y_train = pd.concat([X_train, X_val]), pd.concat([y_train, y_val])
#X_train_rs, y_train_rs = imbalanced_sampling(
#    method='over'
#    , X_train=X_train
#    , y_train=y_train
#)

if ALGORITHM != 'NeuralNetworkClassifier':
    # set algorithm best hyperparams
    hyperparams = (
        config["optimization"]["param_grid"][ALGORITHM]["fixed"]
        | study.best_trial.params
    )

    clf = Classifier(algorithm=ALGORITHM, **hyperparams)
    clf.fit(X=X_train_rs, y=y_train_rs)

    # test set evaluation
    eval = Evaluation(clf=clf, threshold=0.5)
    display(eval.fit(train=(X_train_rs, y_train_rs), test=(X_val, y_val)))

else:
    # get best model and retrain on whole training set
    best_model = bayesian_search_tuner.get_best_models(num_models=1)[0]
    best_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
        )

    history = best_model.fit(
        X_train_rs.to_numpy(), y_train_rs.to_numpy(),
        epochs=5, batch_size=32
    )

    # test set evaluation
    print(best_model.evaluate(X_test.to_numpy(), y_test.to_numpy()))

Epoch 1/5


c:\Users\gonpr\ML_Projects\Fraud\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


16038/16038 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9995 - loss: 0.0028
Epoch 2/5
16038/16038 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.9995 - loss: 0.0024
Epoch 3/5
16038/16038 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.9995 - loss: 0.0025
Epoch 4/5
16038/16038 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - accuracy: 0.9995 - loss: 0.0026
Epoch 5/5
16038/16038 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9995 - loss: 0.0025
446/446 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9994 - loss: 0.0037
[0.004109115805476904, 0.9994382262229919]
